In [1]:
import socket
import subprocess
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time
import subprocess

class DroneControl:
    def send_command(self, command):
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 5555))
        client_socket.send(command.encode())    
        return client_socket
    
    def take_off(self):
        self.send_command('initialize')

    def motor_control(self, front_right_pwm, rear_left_pwm, front_left_pwm, rear_right_pwm, duration):
        self.send_command(f'motor_control {front_right_pwm} {rear_left_pwm} {front_left_pwm} {rear_right_pwm} {duration}')

    def read_imu_data(self):
        client_socket = self.send_command('read_imu_data')
        time_tag, a_x , a_y , a_z , angular_Vel_x , angular_Vel_y , angular_Vel_z  = self.recieved_data(client_socket)
        return time_tag, a_x , a_y , a_z , angular_Vel_x , angular_Vel_y , angular_Vel_z

    def capture_image (self,camera_side):
        if camera_side == 'front':
            client_socket  = self.send_command('render_image front_cam')

        if camera_side == 'right': 
            client_socket  = self.send_command('render_image right_cam')

        if camera_side == 'left':
            client_socket  = self.send_command('render_image left_cam')
        
        image_data = self.recieved_data(client_socket)
        img1d = np.frombuffer(image_data[0], dtype=np.uint8)
        img_rgb = img1d.reshape(image_data[2], image_data[1], 3)
        plt.imshow(img_rgb)

    def recieved_data(self,client_socket):
        # client_socket = self.send_command('recieved_data')
        response = b''
        while True:
            part = client_socket.recv(4096)
            if not part:
                break
            response += part
        client_socket.close()
        if response != b'':
            response = pickle.loads(response)
        return response

In [2]:
cmd_process = subprocess.Popen('cmd', shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
cmd_process.stdin.write(r'cd C:\Users\yaffe\source\repos\AirSim\myenv\Scripts && python API_Commands_Script.py' + '\n')  # Change directory and execute Python script
cmd_process.stdin.flush()
drone = DroneControl()
drone.take_off()

In [ ]:
# Image capturing:
#drone.capture_image('front')
drone.capture_image('right')
#drone.capture_image('left')

In [3]:
drone.read_imu_data()

(1716585383407677952,
 0.03461390361189842,
 -0.037736013531684875,
 -9.383520126342773,
 0.00021310383453965187,
 0.0012228485429659486,
 -0.0012307233409956098)

In [6]:
drone.motor_control(0.02, 0.025, 0.02, 0.025, 5)

In [ ]:
cmd_process = subprocess.Popen('cmd', shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
cmd_process.stdin.write(r'cd C:\Users\yaffe\source\repos\AirSim\myenv\Scripts && python API_Commands_Script.py' + '\n')  
cmd_process.stdin.flush()
